<a href="https://colab.research.google.com/github/wallybeamm/neural_entity_recognition/blob/feature%2Finit/enitity_recognation_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entity Recognotion

## Academic Background

## Practical Implementations

Set parameters

In [6]:
json_path = '/content/test_data.jsonl'
model_name = 'en_core_web_trf'

output_dir = "/content/Model"
n_iter = 100

Install missing packages

In [ ]:
# Restart runtime after installation
!pip install --upgrade --force-reinstall spacy[cuda111]

In [1]:
!pip3 install spacy-transformers
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 4.7 MB 13.7 MB/s 
     |████████████████████████████████| 1.1 MB 55.0 MB/s 
     |████████████████████████████████| 6.6 MB 35.4 MB/s 
     |████████████████████████████████| 163 kB 66.1 MB/s 
2022-10-06 16:46:53.348371: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 32 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


Import librariers

In [4]:
import json
import spacy 
import random
from spacy.training.example import Example


In [10]:
# Load the dataset

def load_dataset(path):
  data = []
  for line in open(path, 'r'):
      line_dict = json.loads(line)
      data.append((line_dict['data'].replace('\n', ' '), line_dict['label']))
  return data

# Display entity info
def show_ents(doc): 
  if doc.ents: 
    for ent in doc.ents: print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
  else: 
    print('No named entities found.')

def train(data, model):
  if "ner" not in model.pipe_names:
      ner = model.create_pipe("ner") # "architecture": "ensemble" simple_cnn ensemble, bow # https://spacy.io/api/annotation
      model.add_pipe(ner)
  else:
      ner = nlp.get_pipe("ner")
  # Update the label list
  for annotations in data:
      for ent in annotations[1]:
          ner.add_label(ent[2])
          

  with model.select_pipes(enable=['transformer', 'ner']):  # only train NER
      optimizer = model.resume_training()
      for itn in range(n_iter):
          random.shuffle(train_data)
          losses = {}
          batches = spacy.util.minibatch(train_data, size=2)
          for batch in batches:
              for text, annotations in batch:
                  # create Example
                  doc = model.make_doc(text)
                  annotations = {'entities' : annotations}
                  example = Example.from_dict(doc, annotations)
                  # Update the model
                  model.update([example], losses=losses, drop=0.3)
  return model

def split(data, train_percantage):
  # Split the data
  train_lenght = int(len(data)*train_percantage)
  train_data = data[:train_lenght]
  test_data = data[train_lenght:]
  return train_data, test_data

def test(test_data, model):
  for text, _ in test_data:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

def same_model(model, output_dir):
  if output_dir is not None:
      nlp.to_disk(output_dir)
      print("Saved model to", output_dir)


In [ ]:
# Main
data = load_dataset(json_path)

nlp = spacy.load(model_name)

train_data, test_data = split(data, 0.7)

finetuned_model = train(train_data, nlp)

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Suggested searches Residential architecture Modern..." with entities "[[185, 196, 'DATE'], [325, 331, 'GPE'], [404, 410,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "MAIN ARTICLES PROFILES ANNOUNCEMENTS EXHIBITIONS A..." with entities "[[371, 375, 'DATE'], [442, 448, 'GPE'], [450, 46

In [ ]:
if output_dir is not None:
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /content/Model
